<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/1_2_EVA_CAMS_GLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verbinden mit der Google Drive und Import der benötigten Module:

In [1]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys

In [3]:
!pip install factor_analyzer
!pip install prophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 849.8 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor_analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42564 sha256=c9fb14f39539c8c4134fbdf43d0b3d319871246d36af091128afc293c5cb1287
  Stored in directory: /root/.cache/pip/wheels/24/59/82/6493618e30ed1cb7a013b9e1b0c9e17de80b04dfcef4ba8a4d
Successfully built factor_analyzer


In [4]:
## import modules

import numpy as np
import pandas as pd
import matplotlib

import seaborn as sns

from datetime import datetime
from datetime import timedelta

In [5]:
latitude = 50.2
longitude = 7.8

seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

## Import:

In [8]:
# Pfad zur CSV-Datei auf Google Drive
# UVI Index Messwerte
name_Messwerte = 'UVI_Messdaten.csv'
# Globalstrahlungsmesswerte
name_Solar_Messwerte = 'Solys_Messdaten.csv'
# CAMS Meteorologische Vorhersagedaten
name_Vorhersage = 'Vorhersage.csv'
# CAMS Globalstrahlungsvorhersage
name_GlobalVorhersage = 'solar_radiation_forecast.csv'

drive_path = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'
output_plot_path = '/content/drive/My Drive/Colab_Notebooks/plot_daily_UVI/'

In [10]:
# Import der Daten für die EDA
df_UVI = pd.read_csv(drive_path + name_Messwerte)
df_Solys = pd.read_csv(drive_path + name_Solar_Messwerte)
df_CAMS = pd.read_csv(drive_path + name_Vorhersage)
df_CAMS_GLO = pd.read_csv(drive_path + name_GlobalVorhersage)

# Messzeitpunkt in Datetime umwandeln
df_UVI.set_index('Datetime', inplace=True)
df_Solys.set_index('Datetime', inplace=True)
df_CAMS.set_index('Datetime', inplace=True)
df_CAMS_GLO.set_index('Observation_period', inplace=True)

# Index in Datetime umwandeln
df_UVI.index = pd.to_datetime(df_UVI.index)
df_Solys.index = pd.to_datetime(df_Solys.index)
df_CAMS.index = pd.to_datetime(df_CAMS.index)
df_CAMS_GLO.index = pd.to_datetime(df_CAMS_GLO.index)

# NaN Einträge durch 0 ersetzen
df_UVI['UVI'] = df_UVI['UVI'].fillna(0)
df_UVI['erythem'] = df_UVI['erythem'].fillna(0)
## df_Solys.fillna(0)

## Beschreibungen der Dataframes:

In [19]:
round(df_CAMS_GLO.describe(), 2).T, df_CAMS_GLO

(                 count    mean    std   min    25%     50%     75%     max
 Clear_sky_GHI  36017.0  135.29  88.59  0.01  64.23  125.11  197.22  310.51
 Clear_sky_BHI  36017.0  104.26  77.03  0.00  42.15   91.62  158.40  270.87
 GHI            36017.0  101.54  80.74  0.01  39.20   79.54  148.72  310.51
 BHI            36017.0   53.55  69.38  0.00   0.29   25.88   84.76  270.87,
                      Clear_sky_GHI  Clear_sky_BHI      GHI       BHI
 Observation_period                                                  
 2022-06-15 07:21:00        188.748        154.357  188.748  154.3570
 2022-06-15 07:23:00        190.480        155.933  190.480  155.9330
 2022-06-15 07:25:00        192.206        157.504  192.206  157.5040
 2022-06-15 07:27:00        193.924        159.068  193.924  159.0680
 2022-06-15 07:29:00        195.635        160.627  195.635  160.6270
 ...                            ...            ...      ...       ...
 2023-03-22 12:19:00        213.539        171.420   77.163

In [16]:
round(df_UVI.describe(), 2).T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,120028.0,60013.50,34649.24,0.00,30006.75,60013.50,90020.25,120027.00
Messzeitpunkt,120028.0,40465.35,13493.43,12360.00,29460.00,40500.00,51480.00,68220.00
erythem,120028.0,0.03,0.04,0.00,0.00,0.02,0.05,0.21
UVI,120028.0,1.33,1.65,0.00,0.15,0.61,1.90,8.54
DiffGreater2,120028.0,0.01,0.10,0.00,0.00,0.00,0.00,1.00
SZA,120028.0,64.29,17.05,27.36,51.10,67.07,77.85,97.87
time_sin,120028.0,0.12,0.70,-1.00,-0.57,0.20,0.83,1.00
time_cos,120028.0,-0.58,0.40,-1.00,-0.92,-0.70,-0.32,0.62
date_sin,120028.0,-0.07,0.70,-1.00,-0.70,-0.25,0.70,1.00
date_cos,120028.0,-0.18,0.69,-1.00,-0.81,-0.33,0.48,1.00


In [17]:
round(df_Solys.describe(), 2).T

,count,mean,std,min,25%,50%,75%,max
Glo,97260.0,254.13,250.23,0.0,52.90,167.03,387.52,1232.83
Dif,97260.0,125.54,109.78,0.0,45.45,91.53,178.38,655.32
Glo_SPLite,97260.0,259.03,248.57,0.0,57.96,173.37,397.06,1189.92
Dir,97260.0,249.70,324.94,0.0,2.04,17.96,531.48,1016.76
Temp,97260.0,16.69,8.59,-7.8,10.60,16.76,23.00,54.32


In [18]:
round(df_CAMS.describe(), 2).T

,count,mean,std,min,25%,50%,75%,max
aod469,4307.0,0.20,0.15,0.02,0.10,0.15,0.26,1.31
aod550,4307.0,0.16,0.12,0.01,0.08,0.13,0.21,1.08
aod670,4307.0,0.12,0.09,0.01,0.06,0.10,0.16,0.82
aod865,4307.0,0.01,0.00,0.00,0.01,0.01,0.01,0.01
uvbed,4307.0,1.26,1.56,-0.00,0.12,0.60,1.83,7.56
uvbedcs,4307.0,1.73,1.91,-0.00,0.21,0.90,2.79,7.67
hcc,4307.0,0.43,0.42,0.00,0.00,0.29,0.94,1.00
lcc,4307.0,0.41,0.40,0.00,0.01,0.28,0.87,1.00
mcc,4307.0,0.32,0.36,0.00,0.00,0.16,0.61,1.00
tcc,4307.0,0.72,0.35,0.00,0.46,0.93,1.00,1.00


## Darstellen der Globalstrahlungsdaten: